In [2]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Embedding, Conv1D,Dense, Dropout

df = pd.read_csv("/home/anushka/Desktop/airplanereviews.csv")

2023-07-28 08:25:33.927196: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-28 08:25:33.946566: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-28 08:25:34.047907: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-28 08:25:34.049139: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-28 08:25:34.623948: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [3]:
review_df = df[['text','airline_sentiment']]


review_df.head(5)

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [4]:
review_df = review_df[review_df['airline_sentiment'] != 'neutral']

print(review_df.shape)
review_df.head(5)

(11541, 2)


,text,airline_sentiment
1,@VirginAmerica plus you've added commercials t...,positive
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
5,@VirginAmerica seriously would pay $30 a fligh...,negative
6,"@VirginAmerica yes, nearly every time I fly VX...",positive


In [5]:
review_df["airline_sentiment"].value_counts()

airline_sentiment
negative    9178
positive    2363
Name: count, dtype: int64

In [6]:
sentiment_label = review_df.airline_sentiment.factorize()
sentiment_label

(array([0, 1, 1, ..., 0, 1, 1]),
 Index(['positive', 'negative'], dtype='object'))

In [7]:
reviews = review_df.text.values

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(reviews)
vocab_size = len(tokenizer.word_index) + 1

In [9]:
encoded_docs = tokenizer.texts_to_sequences(reviews)

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [11]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))

model.add(Conv1D(100, 5, activation='relu'))

model.add(LSTM(100, dropout=0.5, recurrent_dropout=0.5))

model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))




model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [12]:
history = model.fit(padded_sequence,sentiment_label[0], epochs=5, batch_size=32)

Epoch 1/5
361/361 [==============================] - 37s 97ms/step - loss: 0.2999 - accuracy: 0.8783
Epoch 2/5
361/361 [==============================] - 39s 108ms/step - loss: 0.1584 - accuracy: 0.9411
Epoch 3/5
361/361 [==============================] - 42s 116ms/step - loss: 0.0994 - accuracy: 0.9653
Epoch 4/5
361/361 [==============================] - 42s 117ms/step - loss: 0.0697 - accuracy: 0.9745
Epoch 5/5
361/361 [==============================] - 43s 118ms/step - loss: 0.0498 - accuracy: 0.9826


In [14]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(tw).round().item())
    print("Predicted label: ", sentiment_label[1][prediction])


test_sentence1 = "You are looking so pretty today."
predict_sentiment(test_sentence1)

test_sentence2 = "I had a really bad day.."
predict_sentiment(test_sentence2)

1/1 [==============================] - 0s 26ms/step
Predicted label:  positive
1/1 [==============================] - 0s 24ms/step
Predicted label:  negative
